In [9]:
from pandas import DataFrame
from py2neo import Graph,Node,Relationship,NodeMatcher
import pandas as pd
import numpy as np
import os
# 连接Neo4j数据库
graph = Graph('http://localhost:7474/db/data/',username='neo4j',password='123456')

In [11]:
a = Node('Person',name='Tom')
graph.create(a)
b = Node('Person',name='Bob')
graph.create(b)

# 创建关系例子
r = Relationship(a,'KNOWS',b)
graph.create(r)

# 读取节点信息
node = DataFrame(graph.run('MATCH (n:`Person`) RETURN n LIMIT 25'))
# print(node)

# 读取关系信息
relation = DataFrame(graph.run('MATCH (n:`Person`)-[r]->(m:`Person`) return n,m,type(r)'))
# print(relation)

# 删除所有节点
graph.run('MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r')

In [4]:
# 读取数据
stock = pd.read_csv('stock_basic.csv',encoding="gbk")
holder = pd.read_csv('holders.csv')
concept_num = pd.read_csv('concept.csv')
concept = pd.read_csv('stock_concept.csv')
sh = pd.read_csv('sh.csv')
sz = pd.read_csv('sz.csv')
corr = pd.read_csv('corr.csv')

In [5]:
# 数据预处理
stock['行业'] = stock['行业'].fillna('未知')
holder = holder.drop_duplicates(subset=None, keep='first', inplace=False)

In [6]:
# 创建实体（概念、股票、股东、股通）

sz = Node('深股通',名字='深股通')
graph.create(sz)  
 
sh = Node('沪股通',名字='沪股通')
graph.create(sh)  

for i in concept_num.values:
    a = Node('概念',概念代码=i[1],概念名称=i[2])
    print('概念代码:'+str(i[1]),'概念名称:'+str(i[2]))
    graph.create(a)

for i in stock.values:
    a = Node('股票',TS代码=i[1],股票名称=i[3],行业=i[4])
    print('TS代码:'+str(i[1]),'股票名称:'+str(i[3]),'行业:'+str(i[4]))
    graph.create(a)

for i in holder.values:
    a = Node('股东',TS代码=i[0],股东名称=i[1],持股数量=i[2],持股比例=i[3])
    print('TS代码:'+str(i[0]),'股东名称:'+str(i[1]),'持股数量:'+str(i[2]))
    graph.create(a)

In [7]:
# 创建关系（股票-股东、股票-概念、股票-公告、股票-股通）

matcher = NodeMatcher(graph) 
for i in holder.values:    
    a = matcher.match("股票",TS代码=i[0]).first()
    b = matcher.match("股东",TS代码=i[0])
    for j in b:
        r = Relationship(j,'参股',a)
        graph.create(r)
        print('TS',str(i[0]))
            
for i in concept.values:
    a = matcher.match("股票",TS代码=i[3]).first()
    b = matcher.match("概念",概念代码=i[1]).first()
    if a == None or b == None:
        continue
    r = Relationship(a,'概念属于',b)
    graph.create(r) 

noticesdir = os.listdir("notices\\")
for n in noticesdir:
    notice = pd.read_csv("notices\\"+n,encoding="utf_8_sig")
    notice['content'] = notice['content'].fillna('空白')
    for i in notice.values:
        a = matcher.match("股票",TS代码=i[0]).first()
        b = Node('公告',日期=i[1],标题=i[2],内容=i[3])
        graph.create(b)
        r = Relationship(a,'发布公告',b)
        graph.create(r)
        print(str(i[0]))
        
for i in sz.values:
    a = matcher.match("股票",TS代码=i[0]).first()
    b = matcher.match("深股通").first()
    r = Relationship(a,'成分股属于',b)
    graph.create(r)
    print('TS代码:'+str(i[1]),'--深股通')

for i in sh.values:
    a = matcher.match("股票",TS代码=i[0]).first()
    b = matcher.match("沪股通").first()
    r = Relationship(a,'成分股属于',b)
    graph.create(r)
    print('TS代码:'+str(i[1]),'--沪股通')

# 构建股票间关联
corr = pd.read_csv("corr.csv")
for i in corr.values:
    a = matcher.match("股票",TS代码=i[1][:-1]).first()
    b = matcher.match("股票",TS代码=i[2][:-1]).first()
    r = Relationship(a,str(i[3]),b)
    graph.create(r)
    print(i)